In [ ]:
from pathlib import Path
from PIL import Image, ImageDraw
import torch
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov5s.pt')

In [ ]:
def draw_boxes(image_path, detections, save_path=None):
    img = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(img)
    for _, row in detections.iterrows():
        box = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
        draw.rectangle(box, outline="red", width=3)
        draw.text((box[0], box[1]), row['name'], fill="red")
    if save_path:
        img.save(save_path)
    return img

In [ ]:
!pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display

uploader = widgets.FileUpload(accept='image/*', multiple=False)
display(uploader)

In [17]:
!pip install tqdm
import torch
from pathlib import Path
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, messagebox, Label, Button, Frame, ttk
import pandas as pd
import os

# Example model paths (update with your actual paths)
MODEL_PATHS = {
    "YOLOv5": "yolov5s.pt",
    "OBR": "yolov5s.pt",           # Replace with your OBR model path
    "Correlation": "yolov5s.pt"  # Replace with your correlation model path
}

class BottleCounterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Bottle Counter")
        self.root.geometry("540x740")
        self.model = None
        self.image_path = None
        self.result_img = None

        instr = ("Step 1: Select a model/algorithm.\n"
                 "Step 2: Click 'Upload Image' to select a photo.\n"
                 "Step 3: Click 'Count Bottles' to detect bottles.\n"
                 "Step 4: See results and bounding boxes below.")
        self.instr_label = Label(root, text=instr, fg="navy", font=("Arial", 11), justify="left")
        self.instr_label.pack(pady=10)

        # Model selection
        model_frame = Frame(root)
        model_frame.pack(pady=5)
        Label(model_frame, text="Select Model:", font=("Arial", 10)).pack(side=tk.LEFT)
        self.model_var = tk.StringVar(value="YOLOv5")
        self.model_combo = ttk.Combobox(model_frame, textvariable=self.model_var, values=list(MODEL_PATHS.keys()), state="readonly", width=15)
        self.model_combo.pack(side=tk.LEFT, padx=10)
        self.model_combo.bind("<<ComboboxSelected>>", self.on_model_select)

        # Frame for buttons
        btn_frame = Frame(root)
        btn_frame.pack(pady=10)

        self.upload_btn = Button(btn_frame, text="Upload Image", command=self.upload_image, width=15, state=tk.DISABLED)
        self.upload_btn.grid(row=0, column=0, padx=10)

        self.count_btn = Button(btn_frame, text="Count Bottles", command=self.count_bottles, state=tk.DISABLED, width=15)
        self.count_btn.grid(row=0, column=1, padx=10)

        self.exit_btn = Button(btn_frame, text="Exit", command=root.quit, width=10)
        self.exit_btn.grid(row=0, column=2, padx=10)

        # Status and result
        self.status_label = Label(root, text="Select a model to begin.", fg="blue", font=("Arial", 10))
        self.status_label.pack(pady=5)

        self.count_result = Label(root, text="", fg="green", font=("Arial", 14, "bold"))
        self.count_result.pack(pady=5)

        # Canvas for image
        self.img_label = Label(root)
        self.img_label.pack(pady=10)

        # Load default model
        self.load_model()

    def on_model_select(self, event=None):
        self.status_label.config(text="Loading model, please wait...", fg="blue")
        # Reset image and count display
        self.img_label.config(image='')
        self.count_result.config(text="")
        # Disable buttons until a new image is loaded and a model is loaded.
        self.upload_btn.config(state=tk.DISABLED)
        self.count_btn.config(state=tk.DISABLED)
        self.load_model()

    def load_model(self):
        model_name = self.model_var.get()
        model_path = MODEL_PATHS.get(model_name)
        try:
            if model_name == "YOLOv5":
                # Make sure to have the yolov5 directory and its contents available
                self.model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)
            elif model_name == "OBR":
                # Replace with your OBR model loading code
                self.model = torch.load(model_path)
            elif model_name == "Correlation":
                # Replace with your correlation model loading code
                self.model = torch.load(model_path)
            else:
                raise ValueError("Unknown model selected.")
            self.status_label.config(text=f"{model_name} model loaded. Ready to process images.", fg="green")
            self.upload_btn.config(state=tk.NORMAL)
        except Exception as e:
            self.status_label.config(text=f"Model load failed: {e}", fg="red")
            self.upload_btn.config(state=tk.DISABLED)

    def upload_image(self):
        file_path = filedialog.askopenfilename(
            title="Select Image",
            filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")]
        )
        if file_path:
            self.image_path = file_path
            self.display_image(file_path)
            self.count_btn.config(state=tk.NORMAL)
            self.count_result.config(text="")
            self.status_label.config(text="Image loaded. Click 'Count Bottles' to proceed.", fg="blue")

    def display_image(self, img):
        if isinstance(img, str):
            pil_img = Image.open(img)
        elif isinstance(img, Image.Image):
            pil_img = img
        else:
            raise TypeError("Expected a file path (string) or a PIL Image object.")

        pil_img.thumbnail((480, 480))
        self.tk_img = ImageTk.PhotoImage(pil_img)
        self.img_label.config(image=self.tk_img)
        self.img_label.image = self.tk_img

    def count_bottles(self):
        if not self.image_path:
            messagebox.showerror("Error", "Image not loaded.")
            return

        try:
            model_name = self.model_var.get()
            if model_name == "YOLOv5":
                results = self.model(self.image_path)
                
                detections = results.pandas().xyxy[0]
                bottles = detections[detections['name'] == 'bottle']
                count = len(bottles)
                
                rendered_img = results.render()[0]
                self.result_img = Image.fromarray(rendered_img)
                
                self.display_image(self.result_img)
                
            elif model_name == "OBR":
                # Handle OBR detection
                count = 0
                original_img = Image.open(self.image_path)
                detections = pd.DataFrame() # Placeholder
                self.result_img = draw_boxes(original_img, detections)
                self.display_image(self.result_img)
                
            elif model_name == "Correlation":
                # Handle Correlation detection
                count = 0
                original_img = Image.open(self.image_path)
                detections = pd.DataFrame() # Placeholder
                self.result_img = draw_boxes(original_img, detections)
                self.display_image(self.result_img)
                
            else:
                raise ValueError("Unknown model selected.")
                
            self.count_result.config(text=f"Detected {count} bottles.")
            self.status_label.config(text="Detection complete.", fg="green")
            
        except Exception as e:
            messagebox.showerror("Error", f"Failed to process image:\n{e}")
            self.status_label.config(text="Error during detection.", fg="red")
            self.count_result.config(text="")

if __name__ == "__main__":
    root = tk.Tk()
    app = BottleCounterApp(root)
    root.mainloop()

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\User/.cache\torch\hub\master.zip


YOLOv5  2025-8-28 Python-3.10.18 torch-2.8.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [10]:
# bottle_counter_singlepane_with_vps.py
"""
Smart Counting Water Bottle - Single-pane Refined UI
- Single main pane: shows original image initially, detection result after running Count
- Models: Correlation Template Matching (CTM), Canny-Morph-Contour (CMC),
  Watershed (Marker), Vertical Projection Segmentation (VPS), YOLOv5 (optional)
- ROI drawing for CTM: draw rectangle, press Enter to save (rectangle disappears)
- Step outputs shown in a resizable popup (grid of images)
- Uses Vertical Projection Segmentation code integrated from uploaded file
"""

import sys, os, time, random, traceback
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy as np
import cv2
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, messagebox, Label, Button, Frame, ttk, Toplevel, Canvas, Scrollbar

# Optional YOLO (torch must be installed and weights path set in MODEL_PATHS)
try:
    import torch
    TORCH_AVAILABLE = True
except Exception:
    TORCH_AVAILABLE = False

# Put your YOLO weights path here (if you have it). The app will attempt to load if path exists.
MODEL_PATHS = {
    "YOLOv5": "yolov5s.pt",   # change to your weights .pt or leave as-is if you have it in working dir
    "Correlation Template Matching (CTM)": None,
    "Canny-Morph-Contour (CMC)": None,
    "Watershed (Marker)": None,
    "Vertical Projection (VPS)": None
}

# ----------------- Utility image transforms -----------------
def _rand_id(n=6):
    return ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=n))

def laplacian_uint8(bgr):
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 0)
    lap = cv2.Laplacian(blur, cv2.CV_64F)
    lap = np.absolute(lap)
    maxv = lap.max() if lap.max() > 0 else 1.0
    return np.uint8(255.0 * (lap / maxv))

def sobel_mag_uint8(bgr):
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    mag = np.sqrt(gx*gx + gy*gy)
    maxv = mag.max() if mag.max() > 0 else 1.0
    return np.uint8(255.0 * (mag / maxv))

def clahe_gray(bgr):
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return clahe.apply(gray)

def nms_indices(boxes, scores, iou_thresh=0.5):
    if len(boxes) == 0: return []
    boxes = np.array(boxes).astype(float)
    scores = np.array(scores).astype(float)
    x1 = boxes[:,0]; y1 = boxes[:,1]; x2 = boxes[:,0] + boxes[:,2]; y2 = boxes[:,1] + boxes[:,3]
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]
    keep = []
    while order.size > 0:
        i = int(order[0]); keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        w = np.maximum(0.0, xx2 - xx1); h = np.maximum(0.0, yy2 - yy1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter + 1e-8)
        inds = np.where(ovr <= iou_thresh)[0]
        order = order[inds + 1]
    return keep

# ----------------- ROI selector (Enter-to-save rectangle disappears) -----------------
class ROISelector:
    def __init__(self, parent, pil_image, title="Draw ROIs (Enter to save each)"):
        self.parent = parent
        self.orig_pil = pil_image.convert("RGB")
        self.orig_w, self.orig_h = self.orig_pil.size
        self.result_rois = None

        self.win = Toplevel(parent)
        self.win.title(title)
        self.win.transient(parent)
        self.win.grab_set()
        self.win.configure(bg="#ffffff")

        Label(self.win, text="Draw rectangle then press Enter to save (rectangle will disappear). Click Done when finished.",
              bg="#ffffff", fg="#222", font=("Segoe UI", 10)).pack(fill="x", padx=8, pady=(8,2))

        # compute display size (fit to screen)
        screen_w = self.win.winfo_screenwidth() - 200
        screen_h = self.win.winfo_screenheight() - 200
        max_w = min(1100, screen_w)
        max_h = min(800, screen_h)
        scale = min(max_w / self.orig_w, max_h / self.orig_h, 1.0)
        self.disp_w = int(self.orig_w * scale)
        self.disp_h = int(self.orig_h * scale)
        self.scale_x = self.orig_w / max(1, self.disp_w)
        self.scale_y = self.orig_h / max(1, self.disp_h)

        self.disp_pil = self.orig_pil.copy()
        self.disp_pil.thumbnail((self.disp_w, self.disp_h))
        self.tk_img = ImageTk.PhotoImage(self.disp_pil)

        self.canvas = Canvas(self.win, width=self.disp_pil.width, height=self.disp_pil.height, cursor="cross", bg="#f6f7fb")
        self.canvas.pack(padx=8, pady=6)
        self.canvas.create_image(0,0, anchor="nw", image=self.tk_img)

        ctrl = Frame(self.win, bg="#ffffff")
        ctrl.pack(fill="x", padx=8, pady=(6,8))
        self.count_label = Label(ctrl, text="Saved ROIs: 0", bg="#ffffff", font=("Segoe UI", 10, "bold"))
        self.count_label.pack(side="left")
        btn_frame = Frame(ctrl, bg="#ffffff")
        btn_frame.pack(side="right")
        Button(btn_frame, text="Done", command=self.finish, width=10, bg="#4b8bbe", fg="white").pack(side="left", padx=6)
        Button(btn_frame, text="Clear all", command=self.clear_all, width=10, bg="#e0e6ef").pack(side="left", padx=6)
        Button(btn_frame, text="Cancel", command=self.cancel, width=10, bg="#f2a39a").pack(side="left", padx=6)

        self.pending_rect_id = None
        self.pending_coords = None
        self.saved_rois = []

        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<B1-Motion>", self.on_move_press)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)
        self.win.bind("<Return>", self.on_enter_key)
        self.win.bind("<Escape>", lambda e: self.cancel())

        # center window
        self.win.update_idletasks()
        w = self.win.winfo_width(); h = self.win.winfo_height()
        px = int((self.win.winfo_screenwidth() - w) / 2)
        py = int((self.win.winfo_screenheight() - h) / 2)
        self.win.geometry(f"+{px}+{py}")
        self.win.wait_window()

    def on_button_press(self, event):
        x = max(0, min(event.x, self.disp_pil.width - 1))
        y = max(0, min(event.y, self.disp_pil.height - 1))
        if self.pending_rect_id is not None:
            try: self.canvas.delete(self.pending_rect_id)
            except: pass
            self.pending_rect_id = None
            self.pending_coords = None
        self.pending_rect_id = self.canvas.create_rectangle(x, y, x+1, y+1, outline="#ffcf3f", width=2, dash=(4,2))
        self.pending_coords = (x, y, x, y)

    def on_move_press(self, event):
        if self.pending_rect_id is None: return
        cur_x = max(0, min(event.x, self.disp_pil.width - 1))
        cur_y = max(0, min(event.y, self.disp_pil.height - 1))
        x1, y1, _, _ = self.pending_coords
        self.canvas.coords(self.pending_rect_id, x1, y1, cur_x, cur_y)
        self.pending_coords = (x1, y1, cur_x, cur_y)

    def on_button_release(self, event):
        if self.pending_rect_id is None:
            return
        x1,y1,x2,y2 = self.canvas.coords(self.pending_rect_id)
        x1i,y1i,x2i,y2i = int(round(min(x1,x2))), int(round(min(y1,y2))), int(round(max(x1,x2))), int(round(max(y1,y2)))
        w = x2i - x1i; h = y2i - y1i
        if w < 4 or h < 4:
            try: self.canvas.delete(self.pending_rect_id)
            except: pass
            self.pending_rect_id = None; self.pending_coords = None
            return
        self.pending_coords = (x1i, y1i, x2i, y2i)

    def on_enter_key(self, event=None):
        if self.pending_rect_id is None or self.pending_coords is None:
            return
        x1i, y1i, x2i, y2i = self.pending_coords
        ox = int(round(x1i * self.scale_x)); oy = int(round(y1i * self.scale_y))
        ow = int(round((x2i - x1i) * self.scale_x)); oh = int(round((y2i - y1i) * self.scale_y))
        ox = max(0, min(ox, self.orig_w - 1)); oy = max(0, min(oy, self.orig_h - 1))
        if ow < 1: ow = 1
        if oh < 1: oh = 1
        if ox + ow > self.orig_w: ow = self.orig_w - ox
        if oy + oh > self.orig_h: oh = self.orig_h - oy
        self.saved_rois.append((ox, oy, ow, oh))
        try: self.canvas.delete(self.pending_rect_id)
        except: pass
        self.pending_rect_id = None
        self.pending_coords = None
        self.count_label.config(text=f"Saved ROIs: {len(self.saved_rois)}")

    def clear_all(self):
        self.saved_rois = []
        if self.pending_rect_id is not None:
            try: self.canvas.delete(self.pending_rect_id)
            except: pass
        self.pending_rect_id = None
        self.pending_coords = None
        self.count_label.config(text="Saved ROIs: 0")

    def finish(self):
        self.result_rois = list(self.saved_rois)
        self.win.grab_release(); self.win.destroy()

    def cancel(self):
        self.result_rois = None
        self.win.grab_release(); self.win.destroy()

# ----------------- Template Matcher (CTM) -----------------
class TemplateMatcher:
    def __init__(self):
        self.sift_available = False
        try:
            self.sift = cv2.SIFT_create()
            self.sift_available = True
            print("[INFO] SIFT available.")
        except Exception:
            self.sift = None
            self.sift_available = False
            print("[WARN] SIFT not available. Geometric verification disabled (install opencv-contrib-python).")
        self.w_edge = 0.5; self.w_sobel = 0.3; self.w_gray = 0.2

    def gen_sub_templates(self, tpl):
        H,W = tpl.shape[:2]
        res = [(tpl.copy(), {'part':'full'})]
        top_h = max(4, int(H*0.35)); res.append((tpl[0:top_h,:].copy(), {'part':'top'}))
        mid_y = int(H*0.25); mid_h = max(4, int(H*0.5)); res.append((tpl[mid_y:mid_y+mid_h,:].copy(), {'part':'middle'}))
        bot_h = max(4, int(H*0.35)); res.append((tpl[H-bot_h:H,:].copy(), {'part':'bottom'}))
        return res

    def geometric_verify(self, template, image_gray, ratio_test=0.75, min_inliers=6):
        if not self.sift_available or self.sift is None:
            return None
        tpl_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
        detector = self.sift; matcher = cv2.BFMatcher(cv2.NORM_L2)
        kp_t, des_t = detector.detectAndCompute(tpl_gray, None)
        kp_i, des_i = detector.detectAndCompute(image_gray, None)
        if des_t is None or des_i is None or len(kp_t) < 4 or len(kp_i) < 4:
            return None
        try:
            raw = matcher.knnMatch(des_t, des_i, k=2)
        except Exception:
            return None
        good=[]
        for m_n in raw:
            if len(m_n) < 2: continue
            m,n = m_n
            if m.distance < ratio_test * n.distance:
                good.append(m)
        if len(good) < 4: return None
        pts_t = np.float32([kp_t[m.queryIdx].pt for m in good]).reshape(-1,2)
        pts_i = np.float32([kp_i[m.trainIdx].pt for m in good]).reshape(-1,2)
        H, mask = cv2.findHomography(pts_t, pts_i, cv2.RANSAC, 5.0)
        if H is None or mask is None: return None
        inliers = int(mask.sum())
        if inliers < min_inliers: return None
        h_t,w_t = tpl_gray.shape
        corners = np.float32([[0,0],[w_t,0],[w_t,h_t],[0,h_t]]).reshape(-1,1,2)
        mapped = cv2.perspectiveTransform(corners, H).reshape(-1,2)
        x_min = int(max(0, np.min(mapped[:,0]))); y_min = int(max(0, np.min(mapped[:,1])))
        x_max = int(min(image_gray.shape[1]-1, np.max(mapped[:,0]))); y_max = int(min(image_gray.shape[0]-1, np.max(mapped[:,1])))
        w = max(1, x_max - x_min); h = max(1, y_max - y_min)
        conf = min(1.0, float(inliers) / (len(good) + 1e-8))
        return {'bbox':(x_min,y_min,w,h), 'inliers':inliers, 'conf':conf, 'method':'sift'}

    def local_peaks_prominence(self, fused_map, tpl_w, tpl_h, abs_floor=0.3, max_peaks=6, prominence_factor=0.5):
        h,w = fused_map.shape
        k1 = max(3, int(min(tpl_w, tpl_h)/4))
        kernel = np.ones((k1,k1), np.uint8)
        dil = cv2.dilate(fused_map, kernel)
        local_max_mask = fused_map >= dil
        cand = np.argwhere(local_max_mask)
        if cand.size == 0: return []
        neigh = max(5, int(max(tpl_w, tpl_h)/2))
        box = cv2.blur(fused_map.astype(np.float32), (neigh, neigh))
        sq = cv2.blur((fused_map.astype(np.float32)**2), (neigh, neigh))
        local_std = np.sqrt(np.maximum(0.0, sq - box*box))
        peaks=[]
        for (y,x) in cand:
            score = float(fused_map[y,x])
            lmean = float(box[y,x]); lstd = float(local_std[y,x])
            threshold = max(abs_floor, lmean + prominence_factor * lstd)
            if score >= threshold:
                peaks.append((int(x), int(y), score))
        if not peaks: return []
        peaks.sort(key=lambda x: x[2], reverse=True)
        min_dist = max(8, int(min(tpl_w, tpl_h)/3))
        selected=[]
        for (x,y,sc) in peaks:
            too_close=False
            for (sx,sy,ss) in selected:
                if (x-sx)**2 + (y-sy)**2 < (min_dist**2):
                    too_close=True; break
            if too_close: continue
            selected.append((x,y,sc))
            if len(selected) >= max_peaks: break
        return selected

    def _vertical_duplicate_merge(self, detections, y_tol_ratio=0.2):
        if not detections: return []
        out=[]; used=[False]*len(detections)
        for i,a in enumerate(detections):
            if used[i]: continue
            ax,ay,aw,ah = a['bbox']; group=[(i,a)]
            for j,b in enumerate(detections[i+1:], start=i+1):
                if used[j]: continue
                bx,by,bw,bh = b['bbox']
                center_ax = ax + aw/2; center_bx = bx + bw/2
                dx = abs(center_ax - center_bx)
                avg_w = (aw + bw)/2
                yy1 = max(ay, by); yy2 = min(ay+ah, by+bh)
                overlap_v = max(0, yy2 - yy1) / (min(ah,bh) + 1e-8)
                if dx < avg_w*0.35 and overlap_v > y_tol_ratio:
                    group.append((j,b)); used[j]=True
            group.sort(key=lambda x: x[1]['score'], reverse=True)
            out.append(group[0][1])
        return out

    def compute_simple_fused_map(self, image, tpl, weights=(0.5,0.3,0.2)):
        edge_full = laplacian_uint8(image)
        sobel_full = sobel_mag_uint8(image)
        gray_full = clahe_gray(image)
        tpl_edge = laplacian_uint8(tpl)
        tpl_sobel = sobel_mag_uint8(tpl)
        tpl_gray = clahe_gray(tpl)
        th, tw = tpl_edge.shape
        if th >= edge_full.shape[0] or tw >= edge_full.shape[1]:
            return None, None
        try:
            re = cv2.matchTemplate(edge_full, tpl_edge, cv2.TM_CCOEFF_NORMED)
        except Exception:
            re = np.zeros((edge_full.shape[0]-th+1, edge_full.shape[1]-tw+1), dtype=np.float32)
        try:
            rs = cv2.matchTemplate(sobel_full, tpl_sobel, cv2.TM_CCOEFF_NORMED)
        except Exception:
            rs = np.zeros_like(re)
        try:
            rg = cv2.matchTemplate(gray_full, tpl_gray, cv2.TM_CCORR_NORMED)
        except Exception:
            rg = np.zeros_like(re)
        def norm01(a):
            a = a.astype(np.float32); amin = a.min(); amax = a.max()
            if amax - amin < 1e-8: return np.zeros_like(a, dtype=np.float32)
            return (a - amin) / (amax - amin + 1e-8)
        re_n = norm01(re); rs_n = norm01(rs); rg_n = norm01(rg)
        fused = weights[0]*re_n + weights[1]*rs_n + weights[2]*rg_n
        fused = (fused - fused.min()) / (fused.max() - fused.min() + 1e-8)
        fused_u8 = np.uint8(255.0 * fused)
        heat = cv2.applyColorMap(fused_u8, cv2.COLORMAP_JET)
        heat_up = cv2.resize(heat, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_LINEAR)
        overlay = cv2.addWeighted(image, 0.6, heat_up, 0.4, 0)
        pil_overlay = Image.fromarray(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
        return fused, pil_overlay

# ----------------- CTM pipeline wrapper -----------------
def model_compute_match_wrap(counter: TemplateMatcher, sub_tpl, edge_full, sobel_full, gray_full, meta, tpl_limit_params):
    tpl_edge = laplacian_uint8(sub_tpl)
    tpl_sobel = sobel_mag_uint8(sub_tpl)
    tpl_gray = clahe_gray(sub_tpl)
    th, tw = tpl_edge.shape
    if th >= edge_full.shape[0] or tw >= edge_full.shape[1]:
        return []
    try:
        re = cv2.matchTemplate(edge_full, tpl_edge, cv2.TM_CCOEFF_NORMED)
    except Exception:
        return []
    try:
        rs = cv2.matchTemplate(sobel_full, tpl_sobel, cv2.TM_CCOEFF_NORMED)
    except Exception:
        rs = np.zeros_like(re)
    try:
        rg = cv2.matchTemplate(gray_full, tpl_gray, cv2.TM_CCORR_NORMED)
    except Exception:
        rg = np.zeros_like(re)
    def norm01(a):
        a = a.astype(np.float32); amin = a.min(); amax = a.max()
        if amax - amin < 1e-8: return np.zeros_like(a, dtype=np.float32)
        return (a - amin) / (amax - amin + 1e-8)
    re_n = norm01(re); rs_n = norm01(rs); rg_n = norm01(rg)
    fused = counter.w_edge * re_n + counter.w_sobel * rs_n + counter.w_gray * rg_n
    peaks = counter.local_peaks_prominence(fused, tw, th,
                                          abs_floor=tpl_limit_params['min_score_absolute'],
                                          max_peaks=tpl_limit_params['per_template_cap'],
                                          prominence_factor=tpl_limit_params['prominence_factor'])
    out=[]
    for (px,py,sc) in peaks:
        out.append({'bbox':(int(px),int(py),int(tw),int(th)), 'score':float(sc), 'meta':meta})
    return out

def run_ctm_pipeline(counter: TemplateMatcher, image, rois,
                     do_sub_templates=True,
                     do_augmentation=False,
                     per_template_cap=6,
                     min_score_absolute=0.35,
                     prominence_factor=0.5,
                     aspect_ratio_tolerance=0.6,
                     nms_iou=0.5,
                     max_workers=6):
    t0 = time.time()
    edge_full = laplacian_uint8(image)
    sobel_full = sobel_mag_uint8(image)
    gray_full = clahe_gray(image)
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    candidates = []
    pool = ThreadPoolExecutor(max_workers=max_workers)
    futures=[]
    tpl_limit_params = {'per_template_cap':per_template_cap, 'min_score_absolute':min_score_absolute, 'prominence_factor':prominence_factor}
    for tid, (x,y,w,h) in enumerate(rois, start=1):
        tpl = image[y:y+h, x:x+w].copy()
        subs = [(tpl.copy(), {'part':'full'})]
        if do_sub_templates:
            subs = counter.gen_sub_templates(tpl)
        for sub_tpl, meta in subs:
            meta2 = {'template_id':tid, 'scale':1.0, 'rot':0, 'bright':1.0, 'part':meta['part']}
            futures.append(pool.submit(lambda st=sub_tpl, m=meta2: model_compute_match_wrap(counter, st, edge_full, sobel_full, gray_full, m, tpl_limit_params)))
    for f in as_completed(futures):
        try:
            out = f.result()
            for c in out:
                candidates.append(c)
        except Exception as e:
            print("[WARN] worker failed:", e)
    pool.shutdown(wait=True)

    # geometric verification (SIFT-only)
    geom_added=[]
    if counter.sift_available:
        by_tid={}
        for c in candidates:
            tid = c.get('meta',{}).get('template_id', None)
            if tid is None: continue
            by_tid.setdefault(tid, []).append(c)
        for tid, lst in by_tid.items():
            lst.sort(key=lambda x: x['score'], reverse=True)
            cap = min(len(lst), 6)
            orig_tpl = image[rois[tid-1][1]:rois[tid-1][1]+rois[tid-1][3], rois[tid-1][0]:rois[tid-1][0]+rois[tid-1][2]].copy()
            for c in lst[:cap]:
                gm = counter.geometric_verify(orig_tpl, cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), ratio_test=0.75, min_inliers=6)
                if gm is not None:
                    geom_added.append({'bbox':gm['bbox'], 'score':gm['conf'], 'method':gm['method'], 'template_id':tid, 'meta':{'inliers':gm['inliers']}})

    final_candidates = candidates.copy()
    for g in geom_added: final_candidates.append(g)

    filtered=[]
    for c in final_candidates:
        bx,by,bw,bh = c['bbox']; sc = c['score']
        if sc < min_score_absolute: continue
        tid = c.get('meta',{}).get('template_id', None)
        if tid is not None:
            rx,ry,rw,rh = rois[tid-1]
            tpl_ar = rw / (rh + 1e-8)
            cand_ar = bw / (bh + 1e-8)
            ar_ratio = cand_ar / (tpl_ar + 1e-8)
            if ar_ratio < aspect_ratio_tolerance or ar_ratio > (1.0 / aspect_ratio_tolerance):
                continue
        filtered.append(c)

    if len(filtered) == 0:
        return [], 0, None

    boxes = [c['bbox'] for c in filtered]; scores = [c['score'] for c in filtered]
    keep = nms_indices(boxes, scores, iou_thresh=nms_iou)
    final = [filtered[i] for i in keep]
    final.sort(key=lambda x: x['score'], reverse=True)
    final = counter._vertical_duplicate_merge(final, y_tol_ratio=0.25)
    t1 = time.time()
    print(f"[INFO] CTM final detections: {len(final)}. Time: {t1-t0:.2f}s")
    return final, len(final), None

# ----------------- CMC (Canny + Morphology + Contour) -----------------
def classical_detect_and_draw_steps(image_path, min_area=1000, low_thresh=50, high_thresh=150, kernel_size=(5,5)):
    img = cv2.imread(image_path)
    if img is None: raise ValueError("Could not open image.")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, low_thresh, high_thresh)
    kernel = np.ones(kernel_size, np.uint8)
    closing = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    result = img.copy(); bottle_count = 0; detections=[]
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > min_area:
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(result, (x,y), (x+w, y+h), (0,255,0), 2)
            bottle_count += 1
            detections.append({'xmin':x,'ymin':y,'xmax':x+w,'ymax':y+h,'name':'bottle','area':area})
    # do not overlay large text inside image (we'll display count label above separately)
    pil_result = Image.fromarray(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    pil_original = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    pil_edges = Image.fromarray(edges); pil_closing = Image.fromarray(closing)
    import pandas as pd
    detections_df = pd.DataFrame(detections)
    return bottle_count, pil_result, detections_df, pil_original, pil_edges, pil_closing

# ----------------- Watershed (improved preprocessing) -----------------
def watershed_detect_and_draw_steps(image_path, area_threshold=1000, dist_thresh_factor=0.25):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Could not open image.")
    orig = img.copy()
    # Improved preprocessing steps to try to increase Watershed accuracy:
    # - CLAHE for contrast
    # - bilateral filter to reduce noise while preserving edges
    # - morphological opening to remove small objects, then closing
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)
    blur = cv2.bilateralFilter(gray, d=9, sigmaColor=75, sigmaSpace=75)
    # Use adaptive threshold or Otsu. Start with OTSU but fallback to adaptive if necessary.
    try:
        _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    except Exception:
        thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    kernel = np.ones((3,3), np.uint8)
    # opening removes small noise; closing fills small holes
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=2)
    sure_bg = cv2.dilate(closing, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(closing, cv2.DIST_L2, 5)
    # distance threshold can be tuned (lower -> more seeds -> possibly oversegment)
    _, sure_fg = cv2.threshold(dist_transform, dist_thresh_factor * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    num_labels, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0
    markers = cv2.watershed(orig, markers)
    result = orig.copy()
    result[markers == -1] = [0,0,255]
    bottle_count = 0
    detections = []
    for label in np.unique(markers):
        if label <= 1:
            continue
        mask = np.uint8(markers == label)
        area = cv2.countNonZero(mask)
        if area > area_threshold:
            bottle_count += 1
            cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(result, cnts, -1, (0,255,0), 2)
            for cnt in cnts:
                x,y,w,h = cv2.boundingRect(cnt)
                detections.append({'xmin':x,'ymin':y,'xmax':x+w,'ymax':y+h,'name':'bottle','area':area})
    pil_result = Image.fromarray(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    pil_original = Image.fromarray(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))
    pil_thresh = Image.fromarray(thresh)
    pil_closing = Image.fromarray(closing)
    pil_sure_bg = Image.fromarray(sure_bg)
    pil_sure_fg = Image.fromarray(sure_fg)
    import pandas as pd
    detections_df = pd.DataFrame(detections)
    return bottle_count, pil_result, detections_df, pil_original, pil_thresh, pil_closing, pil_sure_bg, pil_sure_fg

# ----------------- Vertical Projection Segmentation (integrated) -----------------
# (Adapted from your uploaded Vertical Projection Segmentation.txt)
PROC_TARGET_HEIGHT = 800
EDGE_MAG_THRESH = 50
COL_SMOOTH_WIN  = 9
COL_MIN_VAL     = 10
GAP_MERGE_PX    = 10
SLICE_MIN_W     = 60
SLICE_MIN_H     = 120

def resize_for_processing(bgr, target_h=PROC_TARGET_HEIGHT):
    H, W = bgr.shape[:2]
    if H == 0 or W == 0:
        raise ValueError("Invalid image dimensions")
    scale = target_h / float(H)
    new_w = max(1, int(round(W * scale)))
    interp = cv2.INTER_AREA if scale < 1.0 else cv2.INTER_CUBIC
    resized = cv2.resize(bgr, (new_w, target_h), interpolation=interp)
    return resized, (1.0/scale)

def preprocess_for_edges(bgr):
    g = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    g = clahe.apply(g)
    blur = cv2.GaussianBlur(g, (0,0), 2.0)
    sharp = cv2.addWeighted(g, 1.8, blur, -0.8, 0)
    gx = cv2.Sobel(sharp, cv2.CV_32F, 1, 0, ksize=3)
    mag = cv2.convertScaleAbs(np.abs(gx))
    return mag

def column_projection(mag, thresh=EDGE_MAG_THRESH, smooth_win=COL_SMOOTH_WIN):
    m = mag.copy()
    m[m < thresh] = 0
    col = m.sum(axis=0).astype(np.float32)
    k = max(3, int(smooth_win)|1)
    kernel = np.ones(k, np.float32)/k
    col_smooth = np.convolve(col, kernel, mode='same')
    return col_smooth

def segments_from_projection(col_smooth, min_val=COL_MIN_VAL, gap_merge=GAP_MERGE_PX):
    W = len(col_smooth)
    mask = (col_smooth >= min_val).astype(np.uint8)
    segs, in_run, start = [], False, 0
    for x in range(W):
        if mask[x] and not in_run:
            in_run, start = True, x
        elif not mask[x] and in_run:
            in_run = False; segs.append([start, x-1])
    if in_run: segs.append([start, W-1])
    if not segs: return []
    merged=[segs[0]]
    for s,e in segs[1:]:
        ps,pe = merged[-1]
        if s - pe - 1 <= gap_merge:
            merged[-1][1] = e
        else:
            merged.append([s,e])
    return merged

def refine_box_from_area(mag, x1, x2, pix_thresh=EDGE_MAG_THRESH):
    slice_mag = mag[:, x1:x2+1]
    m = (slice_mag >= pix_thresh).astype(np.uint8)
    Hs, Ws = m.shape
    min_rows = max(3, int(0.02 * Hs))
    min_cols = max(3, int(0.02 * Ws))
    col_counts = m.sum(axis=0)
    row_counts = m.sum(axis=1)
    good_cols = np.where(col_counts >= min_rows)[0]
    good_rows = np.where(row_counts >= min_cols)[0]
    if good_cols.size >= 2 and good_rows.size >= 2:
        x_min, x_max = good_cols.min(), good_cols.max()
        y_min, y_max = good_rows.min(), good_rows.max()
    else:
        ys, xs = np.where(m)
        if ys.size == 0:
            return x1, 0, x2, mag.shape[0]-1
        x_min, x_max = xs.min(), xs.max()
        y_min, y_max = ys.min(), ys.max()
    X1 = x1 + int(x_min)
    X2 = x1 + int(x_max)
    Y1 = int(y_min)
    Y2 = int(y_max)
    return X1, Y1, X2, Y2

def nms(boxes, iou_thr=0.3):
    if not boxes: return []
    b = np.array(boxes, dtype=np.float32)
    x1,y1,x2,y2 = b[:,0],b[:,1],b[:,2],b[:,3]
    areas = (x2-x1+1)*(y2-y1+1)
    order = areas.argsort()[::-1]
    keep=[]
    while order.size>0:
        i=order[0]; keep.append(i)
        xx1=np.maximum(x1[i],x1[order[1:]])
        yy1=np.maximum(y1[i],y1[order[1:]])
        xx2=np.minimum(x2[i],x2[order[1:]])
        yy2=np.minimum(y2[i],y2[order[1:]])
        w=np.maximum(0.0, xx2-xx1+1)
        h=np.maximum(0.0, yy2-yy1+1)
        inter=w*h
        iou=inter/(areas[i]+areas[order[1:]]-inter+1e-6)
        inds=np.where(iou<=iou_thr)[0]
        order=order[inds+1]
    return [boxes[i] for i in keep]

def boxes_from_vertical_projection(bgr, return_debug=False):
    resized, back_scale = resize_for_processing(bgr, PROC_TARGET_HEIGHT)
    mag = preprocess_for_edges(resized)
    col_s = column_projection(mag)
    segs = segments_from_projection(col_s)
    boxes_norm=[]
    for (x1, x2) in segs:
        if x2 - x1 + 1 < SLICE_MIN_W:
            continue
        X1t, Y1t, X2t, Y2t = refine_box_from_area(mag, x1, x2)
        if (X2t - X1t + 1) < SLICE_MIN_W or (Y2t - Y1t + 1) < SLICE_MIN_H:
            continue
        boxes_norm.append([X1t, Y1t, X2t, Y2t])
    if boxes_norm:
        boxes_norm = nms(boxes_norm, iou_thr=0.3)
    s = back_scale
    boxes_orig = [[int(round(x1*s)), int(round(y1*s)),
                   int(round(x2*s)), int(round(y2*s))] for (x1,y1,x2,y2) in boxes_norm]
    if not return_debug:
        return boxes_orig
    debug = {
        "resized_bgr": resized,
        "mag": mag,
        "segments": segs,
        "boxes_norm": boxes_norm
    }
    return boxes_orig, debug

def build_segmentation_panel(debug):
    resized = debug["resized_bgr"].copy()
    H, W = resized.shape[:2]
    mag   = debug["mag"]
    segs  = debug["segments"]
    boxes = debug["boxes_norm"]
    mag_norm = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    mag_rgb  = cv2.cvtColor(mag_norm, cv2.COLOR_GRAY2BGR)
    seg_vis = resized.copy()
    overlay = seg_vis.copy()
    for (x1, x2) in segs:
        cv2.rectangle(overlay, (x1, 0), (x2, H-1), (0, 255, 255), -1)
    seg_vis = cv2.addWeighted(overlay, 0.25, seg_vis, 0.75, 0)
    for (x1,y1,x2,y2) in boxes:
        cv2.rectangle(seg_vis, (x1,y1), (x2,y2), (0,0,255), 2, cv2.LINE_AA)
    cv2.putText(mag_rgb, "Edge magnitude", (6, 16),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1, cv2.LINE_AA)
    cv2.putText(seg_vis, "Segmentation view", (6, 16),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (50,50,50), 2, cv2.LINE_AA)
    # concat horizontally; if sizes mismatch, pad
    try:
        return cv2.hconcat([mag_rgb, seg_vis])
    except Exception:
        return seg_vis

def draw_boxes(bgr, boxes, color=(0,0,255), thick=2):
    vis = bgr.copy()
    for (x1,y1,x2,y2) in boxes:
        cv2.rectangle(vis, (int(x1),int(y1)), (int(x2),int(y2)), color, thick, cv2.LINE_AA)
    # no large overlay text here (we display count above)
    return vis

# ----------------- Step popup (resizable & grid, no big empty margins) -----------------
def show_steps_popup_images(parent, images_with_titles, title="Step Outputs"):
    popup = Toplevel(parent)
    popup.title(title)
    popup.transient(parent)
    popup.resizable(True, True)
    screen_w = popup.winfo_screenwidth()
    screen_h = popup.winfo_screenheight()
    init_w = int(min(screen_w * 0.85, 1400))
    init_h = int(min(screen_h * 0.85, 900))
    popup.geometry(f"{init_w}x{init_h}")
    outer = Frame(popup)
    outer.pack(fill="both", expand=True)
    canvas = Canvas(outer)
    v_scroll = Scrollbar(outer, orient="vertical", command=canvas.yview)
    h_scroll = Scrollbar(outer, orient="horizontal", command=canvas.xview)
    canvas.configure(yscrollcommand=v_scroll.set, xscrollcommand=h_scroll.set)
    v_scroll.pack(side="right", fill="y")
    h_scroll.pack(side="bottom", fill="x")
    canvas.pack(side="left", fill="both", expand=True)
    inner = Frame(canvas)
    canvas.create_window((0,0), window=inner, anchor="nw")
    n = len(images_with_titles)
    if n <= 2:
        cols = n if n>0 else 1
    elif n <= 4:
        cols = 2
    else:
        cols = 3
    padding = 12
    # choose thumbnail size to fill width neatly (avoid large empty areas)
    thumb_w = max(160, int((init_w - (cols+1)*padding) / cols))
    thumb_h = int(thumb_w * 0.7)
    inner._photo_refs = []
    for idx, (title, pil_img) in enumerate(images_with_titles):
        r = idx // cols; c = idx % cols
        frame = Frame(inner, bd=0, relief="flat", padx=6, pady=6)
        frame.grid(row=r, column=c, padx=8, pady=8, sticky="nsew")
        Label(frame, text=title, font=("Segoe UI", 10, "bold")).pack(anchor="w")
        img_copy = pil_img.copy()
        img_copy.thumbnail((thumb_w, thumb_h))
        tk_img = ImageTk.PhotoImage(img_copy)
        inner._photo_refs.append(tk_img)
        lbl = Label(frame, image=tk_img)
        lbl.pack(padx=2, pady=6)
    for col in range(cols):
        inner.grid_columnconfigure(col, weight=1)
    btn_frame = Frame(popup)
    btn_frame.pack(fill="x", padx=8, pady=6)
    Button(btn_frame, text="Close", command=lambda: (popup.grab_release(), popup.destroy()), width=12, bg="#4b8bbe", fg="white").pack(side="right", padx=6)
    def _on_configure(event=None):
        popup.update_idletasks()
        bbox = canvas.bbox("all")
        if bbox is None:
            bbox = (0,0,init_w,init_h)
        canvas.configure(scrollregion=bbox)
    inner.bind("<Configure>", lambda e: _on_configure())
    popup.bind("<Configure>", lambda e: _on_configure())
    def _on_mousewheel(event):
        if event.state & 0x0001:
            canvas.xview_scroll(int(-1*(event.delta/120)), "units")
        else:
            canvas.yview_scroll(int(-1*(event.delta/120)), "units")
    canvas.bind_all("<MouseWheel>", _on_mousewheel)
    canvas.bind_all("<Button-4>", lambda e: canvas.yview_scroll(-1, "units"))
    canvas.bind_all("<Button-5>", lambda e: canvas.yview_scroll(1, "units"))
    popup.update_idletasks()
    popup.grab_set()
    popup.wait_window()

# ----------------- Draw boxes helper for Pandas DataFrame -----------------
def draw_boxes_pil(pil_img, detections_df, box_color=(0,255,0), thickness=2):
    import numpy as _np
    img_cv = cv2.cvtColor(_np.array(pil_img), cv2.COLOR_RGB2BGR)
    for _, row in detections_df.iterrows():
        x1,y1,x2,y2 = int(row.get('xmin',0)), int(row.get('ymin',0)), int(row.get('xmax',0)), int(row.get('ymax',0))
        cv2.rectangle(img_cv, (x1,y1),(x2,y2), box_color, thickness)
        label = str(row.get('name',''))
        if label:
            cv2.putText(img_cv, label, (x1, max(y1-6,0)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 1, cv2.LINE_AA)
    return Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))

# ----------------- Main GUI: Single-pane refined -----------------
class BottleCounterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Smart Counting Water Bottle")
        self.root.geometry("980x760")
        self.root.configure(bg="#f6f8fb")
        self.root.protocol("WM_DELETE_WINDOW", self.on_exit)

        self.template_matcher = None
        self.image_path = None
        self.orig_bgr = None         # original BGR (kept)
        self.current_display_pil = None  # PIL image currently displayed (original or result)
        self.result_pil = None
        self.yolo_model = None
        self._last_rois = None

        self.canvas_img_id = None
        self.display_scale = 1.0

        self._build_ui()

    def _build_ui(self):
        header = Frame(self.root, bg="#ffffff", bd=0)
        header.pack(fill="x", padx=12, pady=(10,6))
        title = Label(header, text="Smart Counting Water Bottle", font=("Segoe UI", 18, "bold"), bg="#ffffff", fg="#1f4e79")
        title.pack(anchor="w", padx=10, pady=(6,0))
        subtitle = Label(header, text="Choose algorithm → Upload image → Count", font=("Segoe UI", 10), bg="#ffffff", fg="#5a6b7a")
        subtitle.pack(anchor="w", padx=10, pady=(0,8))

        ctrl_card = Frame(self.root, bg="#ffffff", bd=1, relief="groove")
        ctrl_card.pack(fill="x", padx=12, pady=(0,10))
        ctrl = Frame(ctrl_card, bg="#ffffff", pady=8)
        ctrl.pack(fill="x", padx=8)

        Label(ctrl, text="Model:", font=("Segoe UI", 10), bg="#ffffff").pack(side="left", padx=(6,8))
        self.model_var = tk.StringVar(value="Correlation Template Matching (CTM)")
        model_list = ["Correlation Template Matching (CTM)", "Canny-Morph-Contour (CMC)", "Watershed (Marker)", "Vertical Projection (VPS)", "YOLOv5"]
        self.model_combo = ttk.Combobox(ctrl, textvariable=self.model_var, values=model_list, state="readonly", width=36)
        self.model_combo.pack(side="left", padx=(0,8))
        self.model_combo.bind("<<ComboboxSelected>>", self.on_model_select)

        self.upload_btn = Button(ctrl, text="Upload Image", width=14, command=self.upload_image, bg="#2b7bd3", fg="white")
        self.upload_btn.pack(side="left", padx=6)

        self.count_btn = Button(ctrl, text="Count Bottles", width=14, command=self.count_bottles, bg="#2baf6a", fg="white", state="disabled")
        self.count_btn.pack(side="left", padx=6)

        self.exit_btn = Button(ctrl, text="Exit", width=10, command=self.on_exit, bg="#e06666", fg="white")
        self.exit_btn.pack(side="right", padx=10)

        # result card: single panel showing original first then result
        card = Frame(self.root, bg="#ffffff", bd=1, relief="solid")
        card.pack(fill="both", expand=True, padx=12, pady=(0,12))
        topbar = Frame(card, bg="#ffffff")
        topbar.pack(fill="x")
        Label(topbar, text="Detection Result", bg="#ffffff", font=("Segoe UI", 11, "bold")).pack(side="left", padx=8, pady=8)

        # status area above image: left small status + right big Total text
        status_area = Frame(card, bg="#ffffff")
        status_area.pack(fill="x", padx=8)
        self.processing_label = Label(status_area, text="", bg="#ffffff", fg="#666", font=("Segoe UI", 10))
        self.processing_label.pack(side="left", padx=(2,6), pady=6)
        self.total_label = Label(status_area, text="", bg="#ffffff", fg="#2b7bd3", font=("Segoe UI", 14, "bold"))
        self.total_label.pack(side="right", padx=8, pady=6)

        # single canvas with scrollbars
        canvas_outer = Frame(card, bg="#ffffff")
        canvas_outer.pack(fill="both", expand=True, padx=8, pady=(4,8))
        self.canvas = Canvas(canvas_outer, bg="#f6f7fb")
        self.vscroll = Scrollbar(canvas_outer, orient="vertical", command=self.canvas.yview)
        self.hscroll = Scrollbar(canvas_outer, orient="horizontal", command=self.canvas.xview)
        self.canvas.configure(yscrollcommand=self.vscroll.set, xscrollcommand=self.hscroll.set)
        self.canvas.grid(row=0, column=0, sticky="nsew")
        self.vscroll.grid(row=0, column=1, sticky="ns")
        self.hscroll.grid(row=1, column=0, sticky="we")
        canvas_outer.rowconfigure(0, weight=1); canvas_outer.columnconfigure(0, weight=1)
        # mouse wheel to scroll; Ctrl+wheel to zoom
        self.canvas.bind("<MouseWheel>", self._on_canvas_mousewheel)
        self.canvas.bind("<Button-4>", lambda e: self._on_canvas_mousewheel_linux(e, -1))
        self.canvas.bind("<Button-5>", lambda e: self._on_canvas_mousewheel_linux(e, 1))

        # status footer
        status = Frame(self.root, bg="#ffffff")
        status.pack(fill="x", padx=12, pady=(0,10))
        self.status_var = tk.StringVar(value="Ready")
        Label(status, textvariable=self.status_var, anchor="w", bg="#ffffff", fg="#333").pack(side="left", padx=8, pady=6)

        self.on_model_select()

    # ----------------- canvas zoom & scroll -----------------
    def _on_canvas_mousewheel(self, event):
        if event.state & 0x0004:  # Ctrl pressed => zoom
            if event.delta == 0: return
            factor = 1.0 + (event.delta/1200.0)
            self.display_scale = max(0.05, min(6.0, self.display_scale * factor))
            self._render_display_image()
        else:
            if event.delta == 0: return
            self.canvas.yview_scroll(int(-1*(event.delta/120)), "units")

    def _on_canvas_mousewheel_linux(self, event, direction):
        if (event.state & 0x0004):
            factor = 1.0 + (-direction * 0.08)
            self.display_scale = max(0.05, min(6.0, self.display_scale * factor))
            self._render_display_image()
        else:
            self.canvas.yview_scroll(direction, "units")

    # ----------------- render display (fit to canvas without empty margins) -----------------
    def _render_display_image(self):
        # if no image, clear
        self.canvas.delete("all")
        self.canvas_img_id = None
        pil = self.current_display_pil
        if pil is None:
            self.canvas.config(scrollregion=(0,0,0,0))
            return
        # compute available inner size for canvas
        canvas_w = max(200, self.canvas.winfo_width() or 600)
        canvas_h = max(200, self.canvas.winfo_height() or 400)
        # scale image to fit canvas tightly while keeping aspect ratio and avoid empty margins:
        img_w, img_h = pil.width, pil.height
        # Use display_scale as multiplier on the "fit" scale so user can zoom out/in around a fitted size
        fit_scale = min(canvas_w / img_w, canvas_h / img_h)
        final_scale = max(0.01, fit_scale * self.display_scale)
        new_w = int(img_w * final_scale); new_h = int(img_h * final_scale)
        if new_w < 1 or new_h < 1:
            return
        resized = pil.resize((new_w, new_h), Image.LANCZOS)
        self.tk_display_img = ImageTk.PhotoImage(resized)
        self.canvas_img_id = self.canvas.create_image(0,0, anchor="nw", image=self.tk_display_img)
        # set scrollregion to exactly image size — no empty margins shown
        self.canvas.config(scrollregion=(0,0,new_w, new_h))
        # keep reference to avoid garbage collection
        self.canvas._img_ref = self.tk_display_img

    # ----------------- UI actions -----------------
    def on_model_select(self, event=None):
        model_name = self.model_var.get()
        self.status_var.set("Model: " + model_name)
        if model_name == "Correlation Template Matching (CTM)":
            try:
                self.template_matcher = TemplateMatcher()
                if self.template_matcher.sift_available:
                    self.status_var.set("CTM selected — SIFT available")
                else:
                    self.status_var.set("CTM selected — SIFT not available")
            except Exception:
                self.template_matcher = None
                self.status_var.set("CTM init failed")
        elif model_name == "YOLOv5":
            if not TORCH_AVAILABLE:
                self.yolo_model = None
                self.status_var.set("YOLOv5 selected — torch not available")
                messagebox.showwarning("YOLO", "torch not available in this environment. YOLO disabled.")
            else:
                model_path = MODEL_PATHS.get("YOLOv5")
                if model_path is None or not Path(model_path).exists():
                    self.yolo_model = None
                    self.status_var.set("YOLOv5 selected — weights not found")
                else:
                    try:
                        self.status_var.set("Loading YOLO model...")
                        self.root.update_idletasks()
                        self.yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=False)
                        self.status_var.set("YOLOv5 loaded.")
                    except Exception as e:
                        self.yolo_model = None
                        self.status_var.set("YOLO load failed")
                        messagebox.showerror("YOLO load failed", str(e))
        else:
            self.template_matcher = None
            self.status_var.set(model_name + " selected")
        # when switching algorithm, restore original image if present
        if self.orig_bgr is not None:
            self.current_display_pil = Image.fromarray(cv2.cvtColor(self.orig_bgr, cv2.COLOR_BGR2RGB))
            self.display_scale = 1.0
            self._render_display_image()
        self.count_btn.config(state="normal" if self.orig_bgr is not None else "disabled")

    def upload_image(self):
        file_path = filedialog.askopenfilename(title="Select image", filetypes=[("Image Files","*.jpg *.jpeg *.png *.bmp *.tiff")])
        if not file_path:
            return
        bgr = cv2.imread(file_path)
        if bgr is None:
            messagebox.showerror("Error", "Cannot read image file.")
            return
        self.image_path = file_path
        self.orig_bgr = bgr
        self.result_pil = None
        self._last_rois = None
        self.current_display_pil = Image.fromarray(cv2.cvtColor(self.orig_bgr, cv2.COLOR_BGR2RGB))
        self.display_scale = 1.0
        self._render_display_image()
        self.count_btn.config(state="normal")
        self.status_var.set("Image loaded. Choose model and press Count Bottles.")
        self.total_label.config(text="")
        self.processing_label.config(text="")

    def count_bottles(self):
        if self.orig_bgr is None:
            messagebox.showerror("No image", "Please upload an image first.")
            return
        model_name = self.model_var.get()
        self.status_var.set("Processing..."); self.processing_label.config(text="Processing...")
        self.root.update_idletasks()
        try:
            if model_name == "Canny-Morph-Contour (CMC)":
                count, pil_result, detections_df, pil_original, pil_edges, pil_closing = classical_detect_and_draw_steps(self.image_path)
                # Steps popup
                images = [("Original", pil_original), ("Edges (Canny)", pil_edges.convert("RGB")),
                          ("After Morphology", pil_closing.convert("RGB")), ("Detection", pil_result)]
                show_steps_popup_images(self.root, images, title="CMC Steps")
                # show result in main pane
                self.result_pil = pil_result
                self.current_display_pil = self.result_pil
                self.display_scale = 1.0
                self._render_display_image()
                self.total_label.config(text=f"Total Bottles: {count}")
                self.processing_label.config(text="Done")
                self.status_var.set("CMC completed.")

            elif model_name == "Watershed (Marker)":
                # improved watershed (preprocessing included)
                count, pil_result, detections_df, pil_original, pil_thresh, pil_closing, pil_sure_bg, pil_sure_fg = watershed_detect_and_draw_steps(self.image_path, area_threshold=1000, dist_thresh_factor=0.25)
                images = [("Original", pil_original),
                          ("Threshold (Binary)", pil_thresh.convert("RGB") if pil_thresh.mode != "RGB" else pil_thresh),
                          ("After Closing", pil_closing.convert("RGB") if pil_closing.mode != "RGB" else pil_closing),
                          ("Sure Background", pil_sure_bg.convert("RGB") if pil_sure_bg.mode != "RGB" else pil_sure_bg),
                          ("Sure Foreground", pil_sure_fg.convert("RGB") if pil_sure_fg.mode != "RGB" else pil_sure_fg),
                          ("Final detection", pil_result)]
                show_steps_popup_images(self.root, images, title="Watershed Steps")
                self.result_pil = pil_result
                self.current_display_pil = self.result_pil
                self.display_scale = 1.0
                self._render_display_image()
                self.total_label.config(text=f"Total Bottles: {count}")
                self.processing_label.config(text="Done")
                self.status_var.set("Watershed completed.")

            elif model_name == "Vertical Projection (VPS)":
                # run vertical projection segmentation
                boxes, debug = boxes_from_vertical_projection(self.orig_bgr, return_debug=True)
                vis = draw_boxes(self.orig_bgr, boxes, color=(0,255,0))
                pil_vis = Image.fromarray(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
                seg_panel_cv = build_segmentation_panel(debug)
                pil_seg_panel = Image.fromarray(cv2.cvtColor(seg_panel_cv, cv2.COLOR_BGR2RGB))
                images = [("Original", Image.fromarray(cv2.cvtColor(self.orig_bgr, cv2.COLOR_BGR2RGB))),
                          ("Edge magnitude & Segmentation", pil_seg_panel),
                          ("Detection", pil_vis)]
                show_steps_popup_images(self.root, images, title="VPS Steps")
                self.result_pil = pil_vis
                self.current_display_pil = self.result_pil
                self.display_scale = 1.0
                self._render_display_image()
                self.total_label.config(text=f"Total Bottles: {len(boxes)}")
                self.processing_label.config(text="Done")
                self.status_var.set("VPS completed.")

            elif model_name == "Correlation Template Matching (CTM)":
                rois = getattr(self, "_last_rois", None)
                # ask for ROI if not provided
                if rois is None:
                    pil_orig = Image.fromarray(cv2.cvtColor(self.orig_bgr, cv2.COLOR_BGR2RGB))
                    selector = ROISelector(self.root, pil_orig)
                    rois = selector.result_rois
                    if rois is None:
                        self.status_var.set("ROI selection cancelled.")
                        self.processing_label.config(text="Cancelled")
                        return
                    if not rois:
                        messagebox.showinfo("No ROI", "No ROI selected; aborting.")
                        self.status_var.set("No ROI selected.")
                        self.processing_label.config(text="Cancelled")
                        return
                if self.template_matcher is None:
                    self.template_matcher = TemplateMatcher()
                final, count, _ = run_ctm_pipeline(self.template_matcher, self.orig_bgr, rois,
                                                   do_sub_templates=True, do_augmentation=False,
                                                   per_template_cap=6, min_score_absolute=0.35,
                                                   prominence_factor=0.5, aspect_ratio_tolerance=0.6,
                                                   nms_iou=0.5, max_workers=6)
                out_cv = self.orig_bgr.copy()
                for i,m in enumerate(final, start=1):
                    x,y,w,h = m['bbox']; sc = m.get('score',0.0); tid = m.get('template_id', -1)
                    cv2.rectangle(out_cv, (x,y),(x+w,y+h), (0,255,0), 2)
                    cv2.putText(out_cv, f"{i}:T{tid}:{sc:.2f}", (x, max(12,y-6)), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,255,0), 1)
                pil_final = Image.fromarray(cv2.cvtColor(out_cv, cv2.COLOR_BGR2RGB))
                # show steps: include ROI overlay, edges, fused overlay if available
                orig_with_rois = self.orig_bgr.copy()
                for idx,(xx,yy,ww,hh) in enumerate(rois, start=1):
                    cv2.rectangle(orig_with_rois, (xx,yy),(xx+ww,yy+hh),(0,255,0),2)
                    cv2.putText(orig_with_rois, f"ROI{idx}", (xx, max(12,yy-6)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
                pil_orig_rois = Image.fromarray(cv2.cvtColor(orig_with_rois, cv2.COLOR_BGR2RGB))
                pil_edge = Image.fromarray(laplacian_uint8(self.orig_bgr))
                pil_sobel = Image.fromarray(sobel_mag_uint8(self.orig_bgr))
                pil_clahe = Image.fromarray(clahe_gray(self.orig_bgr))
                fused_overlay = None
                try:
                    tpl_roi = self.orig_bgr[rois[0][1]:rois[0][1]+rois[0][3], rois[0][0]:rois[0][0]+rois[0][2]].copy()
                    fused_map, fused_overlay = self.template_matcher.compute_simple_fused_map(self.orig_bgr, tpl_roi, weights=(self.template_matcher.w_edge, self.template_matcher.w_sobel, self.template_matcher.w_gray))
                except Exception:
                    fused_overlay = pil_edge.convert("RGB")
                images = [("Original + ROIs", pil_orig_rois),
                          ("Laplacian (edge)", pil_edge.convert("RGB")),
                          ("Sobel magnitude", pil_sobel.convert("RGB")),
                          ("CLAHE gray", pil_clahe.convert("RGB")),
                          ("Fused score overlay (ROI1)", fused_overlay),
                          ("Final detection", pil_final)]
                show_steps_popup_images(self.root, images, title="CTM Steps")
                self.result_pil = pil_final
                self.current_display_pil = self.result_pil
                self.display_scale = 1.0
                self._render_display_image()
                self.total_label.config(text=f"Total Bottles: {count}")
                self.processing_label.config(text="Done")
                # clear rois if you want (we keep not to force reselect)
                self._last_rois = None
                self.status_var.set("CTM completed.")

            elif model_name == "YOLOv5":
                if not TORCH_AVAILABLE or self.yolo_model is None:
                    # attempt to load if path ok
                    model_path = MODEL_PATHS.get("YOLOv5")
                    if TORCH_AVAILABLE and model_path and Path(model_path).exists():
                        try:
                            self.status_var.set("Loading YOLO model...")
                            self.root.update_idletasks()
                            self.yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=False)
                        except Exception as e:
                            self.yolo_model = None
                            self.status_var.set("YOLO load failed")
                            messagebox.showerror("YOLO load failed", str(e))
                            return
                    else:
                        messagebox.showerror("YOLO Error", "YOLOv5 model not loaded. Check torch and weights configuration.")
                        self.status_var.set("YOLO not available.")
                        return
                results = self.yolo_model(self.image_path)
                detections = results.pandas().xyxy[0]
                # attempt robust filtering for 'bottle'
                if 'name' in detections.columns:
                    bottles = detections[detections['name'].str.lower() == 'bottle']
                else:
                    bottles = detections
                count = len(bottles)
                rendered = results.render()
                if len(rendered) > 0:
                    arr = rendered[0]
                    try:
                        pil_result = Image.fromarray(arr)
                    except Exception:
                        pil_result = Image.fromarray(cv2.cvtColor(arr, cv2.COLOR_BGR2RGB))
                    self.result_pil = pil_result
                    self.current_display_pil = self.result_pil
                    self._render_display_image()
                else:
                    # fallback: draw boxes and show
                    self.result_pil = draw_boxes_pil(Image.open(self.image_path).convert("RGB"), bottles)
                    self.current_display_pil = self.result_pil
                    self._render_display_image()
                self.total_label.config(text=f"Total Bottles: {count}")
                self.processing_label.config(text="Done")
                self.status_var.set("YOLO completed.")

            else:
                self.status_var.set("Unknown model selected.")
                self.processing_label.config(text="")

            messagebox.showinfo("Done", "Detection finished. Result shown in the main pane.")
        except Exception as e:
            traceback.print_exc()
            messagebox.showerror("Error", f"Processing failed:\n{e}")
            self.status_var.set("Error during detection.")
            self.processing_label.config(text="Error")
        finally:
            try:
                self.root.update_idletasks(); self.root.update()
            except:
                pass

    def on_exit(self):
        try: cv2.destroyAllWindows()
        except: pass
        try:
            self.root.quit(); self.root.destroy()
        except:
            try: sys.exit(0)
            except: pass

# ----------------- run -----------------
def main():
    root = tk.Tk()
    app = BottleCounterApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


[INFO] SIFT available.


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\img_asgm\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_24320\2360331169.py", line 788, in _on_mousewheel
    canvas.yview_scroll(int(-1*(event.delta/120)), "units")
  File "C:\Users\User\anaconda3\envs\img_asgm\lib\tkinter\__init__.py", line 1967, in yview_scroll
    self.tk.call(self._w, 'yview', 'scroll', number, what)
_tkinter.TclError: invalid command name ".!toplevel2.!frame.!canvas"
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\img_asgm\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_24320\2360331169.py", line 788, in _on_mousewheel
    canvas.yview_scroll(int(-1*(event.delta/120)), "units")
  File "C:\Users\User\anaconda3\envs\img_asgm\lib\tkinter\__init__.py", lin